<a href="https://colab.research.google.com/github/Akitikori/redditscraper/blob/main/Data_from_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# install praw (python reddit api wrapper)
!pip install praw

In [2]:
#import praw
import praw

# instantiate a reddit client with the client id and secret of the app. I have it hardcoded for now, but will make it more secure in the future
reddit = praw.Reddit(
    client_id = "FZ_7CrhxPuuKg6yJvOYb7g",
    client_secret = "eket0FXt6H3df6y60wy-dyNBzI-mvA",
    user_agent = "afroht data scraping by /u/Akitikori",
)

# check if it's a read only instance
print(reddit.read_only)

True


In [3]:
# Get the hottest topics from the afro HT group
hot_posts = reddit.subreddit('AfroHairTransplant').hot(limit=10)
for post in hot_posts:
    print(post.title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Concerning spam posts and reporting
HEVA Discount & Affliate Link - For those who already decided to go to HEVA - use this link!
Afro hair oral dutasteride + topical minoxidil experience  (2 years post hairline transplant ) 
Spots / Bumps on Donor. How do I get rid of them post HT?
Essentials needed after HT in Turkey?
heva clinic 2600 grafts 3 half months post op 
How long before I can wear a Durag
Stressing out
4700 grafts Day 14 Scab removal - what oils would you recommend? 
What did yall do in the airport to not stick out?


In [3]:
# it seems like asyncpraw is recommended for scripts so I'll import that instead
!pip install asyncpraw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 4.8 MB/s eta 0:00:00


In [4]:
#import async praw
import asyncpraw

reddit = asyncpraw.Reddit(
    client_id = "FZ_7CrhxPuuKg6yJvOYb7g",
    client_secret = "eket0FXt6H3df6y60wy-dyNBzI-mvA",
    user_agent = "afroht data scraping by /u/Akitikori",
)

print(reddit.read_only)

True


In [18]:
# use the await keyword for the async function
afroht_subreddit = await reddit.subreddit("AfroHairTransplant")
# apparently, if you use await then it doesn't load all the information from the subreddit
# you need to use .load() to get more data
await afroht_subreddit.load()
print(f"Title is {afroht_subreddit.title}")
print(f"Description is {afroht_subreddit.public_description}")
print(afroht_subreddit.subscribers)



Title is AfroHairTransplant
Welcome to r/afrohairtransplant – your dedicated space beyond r/HairTransplants! This community blossoms as a specialized corner for those rocking Afro-American, African, and Afro-Caribbean hair textures. Dive into a world where progress shares and questions about hair transplants take center stage, tailored just for our unique hair journey. Let's grow together!
798
AfroHairTransplant


In [40]:
# iterate through as many submissions as you want on the subreddit
n = 1
async for submission in afroht_subreddit.new(limit=n):
    await submission.load()
    print(submission.selftext)
    comments = await submission.comments()
    # replace the 'More comments' link in long threads up to the defined limit
    await comments.replace_more(limit=5)
    # return a list that turns the comment forest structure into a flat list
    # using breadth-first search
    all_comments = await comments.list()
    for comment in all_comments:
        print(comment.body)

Is this much discoloration normal and if so when will it go back to normal. Also when does inflammation go down? Have looked things up obviously but at times people are anxious. 
The discolouration is just because the hair hasn't grown yet and it is healing. Once the hair grows it will blend it with the rest of the hair.
What discolouration do you mean?
how many grafts? which clinic?
But if i cut into this side looks darker
My skin is darker in the transplant area than other places. It makes it look more dense than it is
It just healing bro. Give it few more weeks and you will see it will return to the same colour
How far in the process are you?
Thank you
I just took scabs off I just was making sure that’s normal


In [59]:
# I want to see if I can get the data in a json format which shows the tree level structure of the replies
import json
def get_comment_data(comment):
    return {
        'body': comment.body,
        'upvotes': comment.score,
        'author': comment.author.name if comment.author else 'None',
        'replies': [get_comment_data(reply) for reply in comment.replies]
    }

# redefine submission so that I can use it properly for the next loop
comments_data = []
async for submission in afroht_subreddit.top(limit=50):
    await submission.load()
    await submission.comments.replace_more(limit=5)
    # for each comment, append the author's name, body of the post, and the
    # replies based on the get comment data function defined above
    comments_data.append({
        'title': submission.title,
        'body': submission.selftext,
        'upvotes': submission.score,
        'author': submission.author.name if submission.author else 'None',
        'replies': [get_comment_data(comment) for comment in submission.comments]
    })


with open("comments.json", "w", encoding="utf-8") as jsonfile:
  json.dump(comments_data, jsonfile, ensure_ascii=False, indent=4)